In [1]:
import minsearch
import json

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [6]:
q = "the course has already started, can i still enroll?" #question example

In [7]:
index.fit(documents)

In [8]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI
import os

In [11]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [12]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [13]:
response.choices[0].message.content

"It's never too late to inquire about joining a course! The specific answer to whether it's too late depends on several factors, such as:\n\n1. **Enrollment Deadlines**: Check if the course has a specific enrollment deadline that has already passed.\n2. **Course Type**: Some courses, especially online or self-paced ones, may have rolling admissions or flexible start dates.\n3. **Availability**: Inquire if there are still spots available in the course.\n4. **Institution Policies**: Some institutions allow late enrollment with permission from the instructor or an academic advisor.\n5. **Course Progress**: Consider how far into the term the course is and whether you’ll be able to catch up on missed material.\n\nI recommend contacting the course organizer, instructor, or admissions office for the most accurate and personalized information. They can provide you with any possible options for late enrollment and guidance on how to proceed."

In [14]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [15]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [19]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [20]:
print(response.choices[0].message.content)

Yes, even if the course has already started, you are still eligible to enroll and submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.


## MORE MODULAR

In [21]:
def search(query):
    boost = {
        "question": 3.0, #the question field is 3 times mr important than the other fields
        'section':0.5 # this filed is 0.5 times less important than the other fields
    }

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
        boost_dict=boost,
        num_results=10
    )
    
    return results

In [22]:
# verify the search function
search("How do i run Kafta?")

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial tha

In [23]:
def build_prompt(query,search_results):
    
    prompt_template = """
    You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
    Use only the facts from CONTEXT when answering when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, out NONE

    QUESTION: {question}

    CONTEXT:{context}

    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [24]:
def llm(prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    ) 
    
    return response.choices[0].message.content

In [25]:
query = "how do i run kafka?"
search_results = search(query)
prompt = build_prompt(query,search_results)
answer = llm(prompt)

In [26]:
print(answer)

To run Kafka in the terminal for Java, navigate to the project directory and execute the following command:

```sh
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

This command should help you run the producer/consumer/kstreams/etc. If you encounter further issues, refer to module-specific instructions or guidance.


In [27]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query) # we can replace this other search function (ex: elastic search)
    prompt = build_prompt(query,search_results) #we can optimize the prompt function
    answer = llm(prompt) #we can use another llm model (ex: mistral, ada, etc.)
    
    return answer

In [28]:
rag('The course has already started, can i still enroll?')

'Yes, even if the course has already started, you can still enroll and are eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects to avoid last-minute submissions.'

In [ ]:
# we had this custom search function, because in the future we can replace it with another search function (ex: elastic search) and also we sometimes is not easy to use elastic search with other cloud providers (ex: saturn cloud, etc. - sponsor)